In [1]:
import os
import cv2
import h5py
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
from sklearn.metrics import f1_score

from keras import optimizers
from keras import backend as K
from keras.models import Sequential
from keras.layers import AveragePooling2D, Flatten, Dropout, Dense
from keras.callbacks import TensorBoard
from keras.applications import InceptionV3
from keras.applications import imagenet_utils
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator

verbose = True

os.chdir('/mnt/LSDA/CNN/src')

/home/lsda/lsdaenv/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [3]:
import numpy as np
from keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
class Metrics(Callback):
    def __init__(self, best_val_score):
        super().__init__()
        self.best_val_score = best_val_score
    
    def on_train_begin(self, logs={}):
        self.val_f1s = []
        self.val_recalls = []
        self.val_precisions = []
 
    def on_epoch_end(self, epoch, logs={}):
        val_predict = (np.asarray(self.model.predict(validation_features))).round()
        val_targ = validation_labels
        _val_f1 = f1_score(val_targ, val_predict, average='micro')
        _val_recall = recall_score(val_targ, val_predict, average='micro')
        _val_precision = precision_score(val_targ, val_predict, average='micro')
        self.val_f1s.append(_val_f1)
        self.val_recalls.append(_val_recall)
        self.val_precisions.append(_val_precision)
        print(' — val_f1: %f — val_precision: %f — val_recall %f' %(_val_f1, _val_precision, _val_recall))
        if _val_f1 > self.best_val_score:
            print('Better model found, saving model with val score %s'%_val_f1)
            self.model.save('best_full_model.h5')
            self.best_val_score = _val_f1
        return

In [4]:
FEATURES_FOLDER = '/home/lsda/features'
LABELS_FOLDER = '/mnt/LSDA/labels'

DATASET = 'validation'
f = h5py.File(os.path.join(FEATURES_FOLDER,DATASET,'incept.hdf5')); 
validation_features = f['a'][1:]; f.close()
validation_labels = np.load(os.path.join(LABELS_FOLDER, DATASET, 'labels.npz'))['arr_0'][1:]

In [6]:
# %%============================================================================
# BUILD TOP LAYER
#===============================================================================
# Load files
# NOTE: These files may be too large to fit in memory!

# Load features extracted from the CNN
# f = h5py.File('incept.hdf5')
# CNN_features = f['a']
# f.close()
#CNN_features = np.random.rand(no_imgs_batch,2048) # test construction
# train_data = np.load(open('bottleneck_features_train.npy'))
#if verbose:
#    print('CNN_features shape: ', CNN_features.shape, '\n')

# validation_data = ...
# validation_labels = ...

model_top_layer = Sequential()
# model_top_layer.add(Flatten(input_shape=(1,2048))) # the comma in (2048,) is important!
model_top_layer.add(Dense(1024, activation='sigmoid', input_shape=(2048,), name='dense_1'))
#model_top_layer.add(Dropout(0.5))
model_top_layer.add(Dense(229,
                kernel_initializer='he_uniform',
                activation='sigmoid',
                name='dense_2'))
model_top_layer.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy',f1])

model_top_layer.load_weights('best_val_model.h5')
val_predict = (np.asarray(model_top_layer.predict(validation_features))).round()
_val_f1 = f1_score(val_predict, validation_labels, average='micro')
metrics = Metrics(_val_f1)

tbCallBack = TensorBoard(log_dir='./Tensorboard/logs_%s'%time.time(),
                         histogram_freq=0,
                         write_graph=True,
                         write_images=True)

Instructions for updating:
Use the retry module or similar alternatives.


In [ ]:
# %%============================================================================
# TRAIN TOP LAYER
#===============================================================================
#if verbose:
#    print('Training top layer..')
#
#model_top_layer.fit(CNN_features, y_train,
#          epochs=50,
#          batch_size=32,
#          callbacks=[tbCallBack]) # , validation_data=(validation_data, validation_labels)
# model_top_layer.save_weights('top_layer.h5')

In [8]:
# %%============================================================================
# BUILD FULL MODEL
#===============================================================================
img_size=299
model_full = Sequential()

# Load and freeze (all but last few) InceptionV3 CNN layers
incep_model = InceptionV3(weights="imagenet",
                          include_top=False,
                          input_shape=(img_size, img_size, 3))
# Freeze all but the last inception modules (https://arxiv.org/pdf/1409.4842.pdf)
# To see list of layers, run:
# for i,layer in enumerate(incep_model.layers):
#     print(f'Layer {i}:', layer)
# Last and second to last layers begins in layer 280 and 249, respectively (using 1-indexing)
last_frozen_layer = 279
for layer in incep_model.layers[:last_frozen_layer]:
    layer.trainable = False
for layer in incep_model.layers[last_frozen_layer:]:
    layer.trainable = True

model_full.add(incep_model)
model_full.add(AveragePooling2D(pool_size=(8,8)))
model_full.add(Flatten())
# model_top_layer.load_weights('model_top_layer_weights_path')
model_full.add(model_top_layer)

model_full.compile(optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy',f1])

if verbose:
    print('')
    print('Full model compiled.')
    print('Here are some details regarding the architecture:')
    for i,chunk in enumerate(model_full.layers):
        if i == 0: # unpack CNN
            print('CNN layers:')
            print('First CNN layer input shape: ', chunk.layers[0].input_shape)
            print('...')
            print('Last CNN layer output shape: ', chunk.layers[-1].output_shape, '\n')
        if i == 1: # Flatten layer
            print('Flatten layer:')
            print('Flatten layer input shape: ', chunk.input_shape)
            print('Flatten layer output shape: ', chunk.output_shape, '\n')
        if i == 2: # AveragePooling2D
            print('Average pooling layer:')
            print('AvPool layer input shape: ', chunk.input_shape)
            print('AvPool layer output shape: ', chunk.output_shape, '\n')
        if i == 3: # unpack top_layer
            print('Top layers:')
            print('First top_layer layer input shape: ', chunk.layers[0].input_shape)
            print('...')
            print('Last top_layer layer output shape: ', chunk.layers[-1].output_shape, '\n')
            # for j,layer in enumerate(chunk.layers):
            #     print(f'Layer {j} input shape: ', layer.input_shape)
            #     print(f'Layer {j} output shape: ', layer.output_shape)
    # from keras.utils import plot_model
    # plot_model(model_full, to_file='full_model.png', show_shapes=True) # spits out a flow-chart of the model


Full model compiled.
Here are some details regarding the architecture:
CNN layers:
First CNN layer input shape:  (None, 299, 299, 3)
...
Last CNN layer output shape:  (None, 8, 8, 2048) 

Flatten layer:
Flatten layer input shape:  (None, 8, 8, 2048)
Flatten layer output shape:  (None, 1, 1, 2048) 

Average pooling layer:
AvPool layer input shape:  (None, 1, 1, 2048)
AvPool layer output shape:  (None, 2048) 

Top layers:
First top_layer layer input shape:  (None, 2048)
...
Last top_layer layer output shape:  (None, 229) 



In [9]:
# %%============================================================================
# IMPORT DATA
#===============================================================================
no_labels = 228

train_dir = '../data/raw_images/train/'
image_files = os.listdir(train_dir)
image_files = np.random.permutation(image_files)
no_imgs_tot = len(image_files)

annotations_dir = '../data/labels/train/labels.npz'
train_labels = np.load(annotations_dir)['arr_0']

In [14]:
class Save_model(Callback):
 
    def on_epoch_end(self, epoch, logs={}):
        self.model.save('full_model.h5')
        return
save_model = Save_model()

In [ ]:
no_imgs_batch = 4000 # number of images loaded into memory

for iteration in range(int(len(image_files)/no_imgs_batch+1)):
    print('Iteration: %s'%iteration)
    subset = image_files[(iteration*no_imgs_batch):((iteration+1)*no_imgs_batch)]
    indices = [int(image_file.split('.')[0]) for image_file in subset]
    
    # Load and store features (x_train) in batches
    x_train = np.zeros((no_imgs_batch, img_size, img_size, 3))
    for i,image_file in enumerate(subset):
        try:
            image = load_img(os.path.join(train_dir,image_file), target_size=(img_size, img_size))
            image = img_to_array(image)
            x_train[i,:,:,:] = image
        except:
            print('Failed to load image %s'%image_file)
    #if verbose:
    #    print('Batch of size %s loaded..'%no_imgs_batch)

    # Load and store labels (y_train)
    y_train = train_labels[indices] # full annotations matrix is padded with one zero row and column, and has shape (no_imgs_tot+1,no_labels+1)

    if verbose:
        print('x_train shape: ', x_train.shape)
        print('y_train shape: ', y_train.shape)
        
    # %%============================================================================
    # FINE-TUNE FULL MODEL
    #===============================================================================
    if verbose:
        print('Fine-tuning full model..')

    # Structure data with datagenerator (with augmentation)
    datagen = ImageDataGenerator(rotation_range=10,
                                 width_shift_range=0.15,
                                 height_shift_range=0.15,
                                 zoom_range=0.15,
                                 horizontal_flip=True)

    model_full.fit_generator(datagen.flow(x_train, y_train, batch_size=500),
                             epochs=2,callbacks=[save_model])
                             # steps_per_epoch=no_imgs_batch/32, samples_per_epoch = no_imgs_batch

Iteration: 0
x_train shape:  (4000, 299, 299, 3)
y_train shape:  (4000, 229)
Fine-tuning full model..
Epoch 1/2
8/8 [==============================] - 84s 10s/step - loss: 24.4383 - acc: 0.1190 - f1: 0.3674
Epoch 2/2
8/8 [==============================] - 77s 10s/step - loss: 24.1095 - acc: 0.1307 - f1: 0.3795
Iteration: 1
Failed to load image 679994.jpg
x_train shape:  (4000, 299, 299, 3)
y_train shape:  (4000, 229)
Fine-tuning full model..
Epoch 1/2
8/8 [==============================] - 84s 11s/step - loss: 24.3993 - acc: 0.1283 - f1: 0.3834
Epoch 2/2
8/8 [==============================] - 80s 10s/step - loss: 24.2856 - acc: 0.1280 - f1: 0.3861
Iteration: 2
Failed to load image 679932.jpg
Failed to load image 679873.jpg
x_train shape:  (4000, 299, 299, 3)
y_train shape:  (4000, 229)
Fine-tuning full model..
Epoch 1/2
8/8 [==============================] - 86s 11s/step - loss: 24.0157 - acc: 0.1338 - f1: 0.3977
Epoch 2/2
8/8 [==============================] - 80s 10s/step - loss: 23.